<a href="https://colab.research.google.com/github/msrjdin/Transformers/blob/master/GPT-2/GPT_Text_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pwd

'/content'

In [2]:
import os

In [3]:
os.mkdir("DS")

In [4]:
os.chdir("DS")

In [5]:
os.mkdir('openAI')

In [6]:
os.chdir("openAI")

In [7]:
pwd

'/content/DS/openAI'

In [8]:
!git clone https://github.com/openai/gpt-2.git


Cloning into 'gpt-2'...
remote: Enumerating objects: 230, done.
remote: Total 230 (delta 0), reused 0 (delta 0), pack-reused 230
Receiving objects: 100% (230/230), 4.38 MiB | 13.19 MiB/s, done.
Resolving deltas: 100% (118/118), done.


In [9]:
os.chdir("gpt-2")

In [10]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [11]:
import tensorflow
print(tensorflow.__version__)

1.15.2


In [12]:
!python3 download_model.py 345M

Fetching checkpoint: 1.00kit [00:00, 726kit/s]                                                      
Fetching encoder.json: 1.04Mit [00:00, 42.7Mit/s]                                                   
Fetching hparams.json: 1.00kit [00:00, 891kit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 1.42Git [00:24, 58.9Mit/s]                                 
Fetching model.ckpt.index: 11.0kit [00:00, 7.85Mit/s]                                               
Fetching model.ckpt.meta: 927kit [00:00, 26.7Mit/s]                                                 
Fetching vocab.bpe: 457kit [00:00, 33.6Mit/s]                                                       


In [14]:
!export PYTHONIOENCODING=UTF-8

In [15]:
os.chdir('src')


In [16]:
import json
import os
import numpy as np
import tensorflow as tf
import model, sample, encoder

In [17]:
def interact_model(
    model_name,
    seed,
    nsamples,
    batch_size,
    length,
    temperature,
    top_k,
    models_dir
):
    models_dir = os.path.expanduser(os.path.expandvars(models_dir))
    if batch_size is None:
        batch_size = 1
    assert nsamples % batch_size == 0

    enc = encoder.get_encoder(model_name, models_dir)
    hparams = model.default_hparams()
    with open(os.path.join(models_dir, model_name, 'hparams.json')) as f:
        hparams.override_from_dict(json.load(f))

    if length is None:
        length = hparams.n_ctx // 2
    elif length > hparams.n_ctx:
        raise ValueError("Can't get samples longer than window size: %s" % hparams.n_ctx)

    with tf.Session(graph=tf.Graph()) as sess:
        context = tf.placeholder(tf.int32, [batch_size, None])
        np.random.seed(seed)
        tf.set_random_seed(seed)
        output = sample.sample_sequence(
            hparams=hparams, length=length,
            context=context,
            batch_size=batch_size,
            temperature=temperature, top_k=top_k
        )

        saver = tf.train.Saver()
        ckpt = tf.train.latest_checkpoint(os.path.join(models_dir, model_name))
        saver.restore(sess, ckpt)

        while True:
            raw_text = input("Model prompt >>> ")
            while not raw_text:
                print('Prompt should not be empty!')
                raw_text = input("Model prompt >>> ")
            context_tokens = enc.encode(raw_text)
            generated = 0
            for _ in range(nsamples // batch_size):
                out = sess.run(output, feed_dict={
                    context: [context_tokens for _ in range(batch_size)]
                })[:, len(context_tokens):]
                for i in range(batch_size):
                    generated += 1
                    text = enc.decode(out[i])
                    print("=" * 40 + " SAMPLE " + str(generated) + " " + "=" * 40)
                    print(text)
            print("=" * 80)

In [ ]:
interact_model(
    '345M',
    1,
    1,
    1,
    100,
    0.9,
    40,
    '/content/DS/openAI/gpt-2/models'
)





Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use `tf.random.categorical` instead.
INFO:tensorflow:Restoring parameters from /content/DS/openAI/gpt-2/models/345M/model.ckpt
======================================== SAMPLE 1 ========================================
 I have been part of a large engineering team, having started my career as a data analyst back in 2008.


How did you get involved with The Big Data Institute (TBI)?


I started working in the field of data science four years ago, at a time when the technology for Big Data was still developing and there was very scant research into the use case. The Big Data Institute is a great resource for anyone looking for an opportunity to get into the area and try their hand at
